# HSV颜色检测
* 通过机器人的摄像头识别矩形框中的HSV颜色值

## 1.导入所需模块

In [ ]:
import cv2
import numpy as np
import math
import os
import sys
import time
from PIL import Image, ImageDraw,ImageFont
import threading
from nxbot import Robot,event
from IPython.display import display
import ipywidgets.widgets as widgets
from nxbot import Robot,event,bgr8_to_jpeg
import traitlets
from traitlets.config.configurable import Configurable
import ipywidgets

## 2.创建可视化小部件

In [ ]:
rbt = Robot()
image_widget = widgets.Image(format='jpeg')
mask_widget = widgets.Image(format='jpeg')

## 3.定义计算HSV均值模块

In [ ]:
def mean_hsv(img,HSV_value):
	HSV = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
	HSV_value[0]+=np.mean(HSV[:, :, 0])
	HSV_value[1]+=np.mean(HSV[:, :, 1])
	HSV_value[2]+=np.mean(HSV[:, :, 2])
	return HSV_value

## 4.定义计算HSV阈值模块

In [ ]:
def hsv_range(HSV_value):
	# 设置HSV颜色值的范围
	H_range = 5 # 色调
	S_range = 40 # 饱和度
	V_range = 100 # 明亮

	lower_H = int(HSV_value[0] - H_range)
	upper_H = int(HSV_value[0] + H_range)

	lower_S = int(HSV_value[1] - S_range)
	upper_S = int(HSV_value[1] + S_range)

	lower_V = int(HSV_value[2] - V_range)
	upper_V = int(HSV_value[2] + V_range)

	if lower_H<0:
		lower_H=0
	if upper_H>180:
		upper_H=180

	if lower_S<30:
		lower_S=30
	if upper_S>255:
		upper_S=255

	if lower_V<10:
		lower_V=10
	if upper_V>255:
		upper_V=255

	lower_HSV = np.array([lower_H, lower_S, lower_V])
	upper_HSV = np.array([upper_H, upper_S, upper_V])
	return lower_HSV, upper_HSV



## 5.定义保存HSV阈值数据模块

In [ ]:
def save_hsv(lower_HSV, upper_HSV):
    content ="HSV颜色范围值为 :" + str(lower_HSV[0]) + ',' +str(lower_HSV[1])+ ','\
                + str(lower_HSV[2])+ ' ' + str(upper_HSV[0])+ ',' + str(upper_HSV[1])+ ',' + str(upper_HSV[2])
    # 将HSV值写入文件HSV_value.txt
    filename = "HSV_value.txt"
    with open(filename, "w") as f:
        f.write(content)
    print("HSV颜色范围值保存在文件" + filename)
    print(content)


## 6.定义测试提取HSV值是否正确模块

In [ ]:
def test(lower_HSV, upper_HSV, image):
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(hsv, lower_HSV, upper_HSV)
    mask = cv2.erode(mask, None, iterations=2)
    mask = cv2.dilate(mask, None, iterations=2)
    mask = cv2.GaussianBlur(mask, (5, 5), 0)
    cv2.putText(mask, 'success!', (20, 30), cv2.FONT_HERSHEY_SIMPLEX, 1,(255, 255, 255), 2, cv2.LINE_AA)
    mask_widget.value = bgr8_to_jpeg(cv2.resize(mask,(320,240)))

## 7.定义获取机器人摄像头图像线程模块
获取机器人图像数据并提取HSV阈值

In [ ]:
HSV_value = [0,0,0]

count = 0

collect_times = 50

def collection():
    global detect_flag
    detect_flag = True
    global HSV_value
    global count
    global collect_times
    lower_HSV = []
    upper_HSV = []
    while detect_flag:
        time.sleep(0.05)
        img = rbt.camera.read()
        if img is not None:
#             img = cv2.flip(img,1)
            # 150次以内先做提醒，将颜色块放在矩形框中
            if count < 150:
                cv2.putText(img, 'put it in the box!', (20, 30), cv2.FONT_HERSHEY_SIMPLEX, 1,
                            (0, 255, 0), 2, cv2.LINE_AA)
            # 如果在150-200次以内开始收集hsv值，并求出平均值
            elif count > 150 and count < 150+collect_times:
                cv2.putText(img, 'collecting!', (20, 30), cv2.FONT_HERSHEY_SIMPLEX, 1,
                            (0, 255, 0), 2, cv2.LINE_AA)
                frame = img[150-20:150+20, 150-20:150+20]
                HSV_value = mean_hsv(frame, HSV_value)
            # 开始检测，查看是否有提取到矩形框中颜色的HSV值
            elif count==150+collect_times:
                for i in range(len(HSV_value)):
                    HSV_value[i] = HSV_value[i] / collect_times
                lower_HSV, upper_HSV = hsv_range(HSV_value)
                save_hsv(lower_HSV, upper_HSV)
            
            elif count>150+collect_times:
                test(lower_HSV, upper_HSV, img)
            count += 1
            img = cv2.rectangle(img, (120, 120), (170,170), (0, 255, 0), 3)
            image_widget.value = bgr8_to_jpeg(cv2.resize(img,(320,240)))
# 创建线程
process1 = threading.Thread(target=collection,)
# 启动线程
process1.start()

## 8.连接机器人开始提取

In [ ]:
rbt.connect()
rbt.camera.start()
rbt.base.set_ptz(0)
display(widgets.HBox([image_widget,mask_widget]))

## 9.断开与机器人的连接

In [ ]:
# detect_flag=False
# rbt.disconnect()